# Profitability - Inflation-based bonds with coupon payment

In [1]:
import numpy as np

# Sobre as taxas https://www.tesourodireto.com.br/conheca/regras.htm
class titulo_ipca_cp(object):
    
    def __init__(self):
        self.vf = 1 # valor de face, o valor que será recebido no vencimento por cada título comprado
        self.tx_c = 6
   
    def get_pv(self, list_du, tx, vna):
        list_num = len(list_du)*[(1 + self.tx_c/100)**(1/2) - 1]
        list_num[-1] += 1
        list_pv = []
        for du, num in zip(list_du, list_num):
            temp_pv = num/(1 + tx/100)**(du/252)
            list_pv.append(temp_pv)
        pv = vna*np.sum(list_pv)
        return pv, list_pv
    
    def venda_antecipada(self, list_du, list_vna, ipca, tx_compra, vnac, vnacv, tx_venda, du_venda, tipo='r'):
        list_num = len(list_du)*[(1 + self.tx_c/100)**(1/2) - 1]
        list_num[-1] += 1
        list_pv = []
        list_cup = []
        pv_compra,_ = titulo_ipca_cp().get_pv(list_du, tx_compra, vnac)
        for du, num, vna in zip(list_du, list_num, list_vna):
            if du_venda < du:
                temp_pv = num/(1 + tx_venda/100)**((du - du_venda)/252)
                list_pv.append(temp_pv)
            else:
                list_cup.append(vna*num*((1+ipca/100)**((du_venda-du)/252))*(1 + tx_venda/100)**((du_venda - du)/252) if tipo == 'r' else num)
        pv_venda = vnacv*np.sum(list_pv)
        valor_bruto_resgate = np.sum(list_cup) + pv_venda
        rb = valor_bruto_resgate/pv_compra - 1
        rba = (1 + rb)**(252/du_venda) - 1 
        
        return pv_venda, rb*100, rba*100

## **Example 1**

In [2]:
vnac = 2506.66
vnacv = 2752.31
tx_compra = 6.10
list_du = [127, 250, 374, 500]
list_vna = [2627.817310, 2752.317192, 0, 0]
tit = titulo_ipca_cp()
du_venda = 250
tx_venda = 6.10
ipca = 9.7
pv_venda, rb, rba = tit.venda_antecipada(list_du, list_vna, ipca, tx_compra, vnac, vnacv, tx_venda, du_venda)

print("R$" + str(np.round(pv_venda,2)))
print(str(np.round(rb,2)) + " %")
print(np.round(rba,2), "%")

R$2751.05
16.44 %
16.58 %
